In [1]:
# %pip install ensemble_boxes

In [2]:
from ensemble_boxes import *
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import json

In [3]:
with open('../dataset/test.json','r') as f:
	testjson=json.load(f)
print(testjson['info'])

cnt=0
for i in testjson['images']:
	print(i['file_name'])
	cnt+=1
	if cnt>3:
		break

{'year': 2021, 'version': '1.0', 'description': 'Recycle Trash', 'contributor': 'Upstage', 'url': None, 'date_created': '2021-02-02 01:10:00'}
test/0000.jpg
test/0001.jpg
test/0002.jpg
test/0003.jpg


In [4]:
# dataset_path='../dataset'
csv_list=[
    '../output/sota_list/sub1.csv',
    '../output/sota_list/sub2.csv',
    '../output/sota_list/sub3.csv',
    '../output/sota_list/sub4.csv',
    '../output/sota_list/sub5.csv',
    '../output/sota_list/sub6.csv',
    '../output/sota_list/submission_latest_rcnn040506_rpniou06.csv',
    '../output/sota_list/submission_yolov5_kfold1.csv',
    '../output/sota_list/swin-b_rpn0.6_24ep.csv',
    '../output/sota_list/yolov5kfold3.csv'
]

In [5]:
iou_thr=0.68
skip_box_thr=0.0001
sigma=0.1


ensemble_str=[]
file_path=[]
for i,test_img in enumerate(testjson['images']):
	
	test_img_id=test_img['file_name']
	total_boxes_list=[]
	total_scores_list=[]
	total_labels_list=[]

	for csv_path in csv_list:
		submission_df=pd.read_csv(csv_path)
		img_id=submission_df['image_id'].iloc[i]
		pred_str=submission_df['PredictionString'].iloc[i]
		
		img_out=pred_str
		boxes_list=[]
		scores_list=[]
		labels_list=[]
		
		if pd.isnull(img_out): continue

		img_out=np.reshape(img_out.split(),(-1,6)).tolist()

		for o in img_out:
			label=int(o[0])
			score=float(o[1])
			box=list(map(float,[o[2],o[3],o[4],o[5]]))
			box=list(map(lambda x:x/1024,box))
			boxes_list.append(box)
			scores_list.append(score)
			labels_list.append(label)
		
		total_boxes_list.append(boxes_list)
		total_scores_list.append(scores_list)
		total_labels_list.append(labels_list)
	

	# boxes, scores, labels = nms(total_boxes_list, total_scores_list, total_labels_list, iou_thr=iou_thr)
	# boxes, scores, labels = soft_nms(total_boxes_list,total_scores_list, total_labels_list, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
	# boxes, scores, labels = non_maximum_weighted(total_boxes_list,total_scores_list, total_labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
	boxes, scores, labels = weighted_boxes_fusion(total_boxes_list,total_scores_list, total_labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)

	temp_str=''
	for box,score,label in zip(boxes,scores,labels):
		temp_str+=str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * 1024) + ' ' + str(box[1] * 1024) + ' ' + str(box[2] * 1024) + ' ' + str(box[3] * 1024) + ' '
			
	ensemble_str.append(temp_str)
	file_path.append(test_img_id)
			


/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.80712890625, 0.0, 0.98291015625].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.0, 0.0, 0.25634765625].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.568359375, 0.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.77783203125, 0.0, 0.99365234375].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_

In [6]:
total_boxes_list[:5]

[[[0.36669921875, 0.200927734375, 0.6103515625, 0.822265625],
  [0.57568359375, 0.5380859375, 0.60986328125, 0.736328125],
  [0.59521484375, 0.014404296875, 0.68603515625, 0.5302734375],
  [0.60595703125, 0.62451171875, 1.0, 0.99755859375],
  [0.00830078125, 0.001220703125, 0.685546875, 0.515625],
  [0.362060546875, 0.4052734375, 0.61181640625, 0.61328125],
  [0.2347412109375, 0.802734375, 0.29345703125, 0.8330078125],
  [0.070556640625, 0.0, 0.66015625, 0.83203125],
  [0.0, 0.00048828125, 0.69775390625, 0.63623046875],
  [0.0, 0.0, 0.000885009765625, 0.9638671875],
  [0.3681640625, 0.5322265625, 0.4638671875, 0.7412109375]],
 [[0.3633314453125, 0.201101943359375, 0.6126943359375, 0.819037734375]],
 [[0.365607353515625, 0.20131171875, 0.614491015625, 0.81802337890625]],
 [[0.36391216796875, 0.201970263671875, 0.6142880859375, 0.819871875]],
 [[0.234924931640625, 0.802488623046875, 0.28368806640625, 0.83364921875],
  [0.364673671875, 0.198755205078125, 0.61257875, 0.81951181640625]]]

In [7]:
submission = pd.DataFrame()

submission['PredictionString'] = ensemble_str
submission['image_id'] = file_path
submission.to_csv('submission_ensemble_soft_nms.csv',index=False)

submission.head()

,PredictionString,image_id
0,7 0.9967287182807922 603.1417846679688 518.348...,test/0000.jpg
1,5 0.9221264719963074 133.7111053466797 0.26945...,test/0001.jpg
2,1 0.9804110527038574 775.91015625 415.77001953...,test/0002.jpg
3,9 0.9885166883468628 157.54530334472656 263.97...,test/0003.jpg
4,1 0.904869019985199 203.34622192382812 257.791...,test/0004.jpg
